In [ ]:
# define sensor and image collection function
from typing import List, Literal
from airgen import ImageType, VehicleClient
from airgen.utils.visualize import rr_log_airgen_image
import cv2
import numpy as np
from grid import GRID_USER_SESSION_BLOB_DIR

def data_capture(car: VehicleClient, sensor_types: List[str], capture_types: Literal['rgb', 'depth', 'segmentation', 'visdepth'], idx: int = 0):
    frame_data = {"sensor_data": {}, "image_data": {}}
    capture_name_map = {"rgb": ImageType.Scene, "depth": ImageType.DepthPerspective, "segmentation": ImageType.Segmentation, "visdepth": ImageType.DepthVis}

    # get sensor data
    for sensor_name in sensor_types:
        if sensor_name == "imu":
            frame_data["sensor_data"][sensor_name] = car.getImuData()
        else:
            raise ValueError(f"Unknown sensor name: {sensor_name}")
    image_types = [capture_name_map[capture_type] for capture_type in capture_types]
    images = car.getImages("front_center", image_types)

    # Received data is a tuple of the image and the corresponding pose
    for (capture_type, image) in zip(capture_types, images):
        #frame_data["image_data"][capture_type] = image[0]
        if capture_type.startswith("depth"):
            img_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, capture_type + "_" + str(idx) + ".npy")
            np.save(img_path, image[0])
        else:
            img_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, capture_type + "_" + str(idx) + ".png")
            cv2.imwrite(img_path, image[0])
        rr_log_airgen_image("grid", capture_name_map[capture_type], image[0])
    return frame_data

data = data_capture(airgen_car_0.client, ['imu'], ['rgb', 'depth', 'segmentation', 'visdepth'])

In [ ]:
from airgen import CarControls
import time

car_controls = CarControls()
car_controls.throttle = 0.5
airgen_car_0.client.setCarControls(car_controls)

for i in range(10):
    data = data_capture(airgen_car_0.client, ['imu'], ['rgb', 'depth', 'segmentation'], i)
    time.sleep(0.5)

airgen_car_0.client.setCarControls(CarControls())

In [ ]:
# save collected data to blob storage
import json
import numpy as np
def serialize_to_dict(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if hasattr(obj, "__dict__"):
        return obj.__dict__
    else:
        raise TypeError(f"Object of type {type(obj)} is not serializable")
data_str = json.dumps(data, default=serialize_to_dict, ensure_ascii=False, indent=4)

# any file in the directory of GRID_USER_SESSION_BLOB_DIR will be uploaded to the blob storage
file_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, 'data_collection_sim.json')
with open(file_path, 'w') as f:
    f.write(data_str)

In [ ]:
airgen_car_0.client.simRunConsoleCommand("t.maxFPS 15")
airgen_car_0.client.enableApiControl(False)